<a href="https://colab.research.google.com/github/s2t2/tweet-analysis-2021/blob/main/notebooks/Big_Query_Connection_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to query a BigQuery database directly from a Colab notebook.

# Setup

## Mounting the Drive

In [74]:
import os

from google.colab import drive

drive.mount('/content/drive')
print(os.listdir(os.getcwd())) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['.config', 'drive', 'sample_data']


In [75]:
DIRPATH = '/content/drive/My Drive/Research/Tweet Analysis 2021'
print(DIRPATH)
os.path.isdir(DIRPATH)

/content/drive/My Drive/Research/Tweet Analysis 2021


True

## Configuring Credentials 


In [76]:
# google.cloud checks the file at path designated by the GOOGLE_APPLICATION_CREDENTIALS env var
# so we set it here using the shared credentials JSON file from our shared google drive
# and verify it for good measure

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(DIRPATH, "credentials", "tweet-research-shared-268bbccc0aac.json") 

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") # implicit check by google.cloud
print(GOOGLE_APPLICATION_CREDENTIALS) # verification for implicit check
print(os.path.isfile(GOOGLE_APPLICATION_CREDENTIALS)) # verification for implicit check

/content/drive/My Drive/Research/Tweet Analysis 2021/credentials/tweet-research-shared-268bbccc0aac.json
True


# Helpers

### BigQuery Service

In [77]:

from google.cloud import bigquery

class BigQueryService():
    def __init__(self):
        self.client = bigquery.Client()

    def execute_query(self, sql, verbose=True):
        if verbose == True: 
            print(sql)
        job = self.client.query(sql)
        return job.result()


In [78]:
bq_service = BigQueryService()
print(bq_service)

## Number Decorators

In [79]:
def fmt_n(large_number):
    """
    Formats a large number with thousands separator, for printing and logging.

    Param large_number (int) like 1_000_000_000

    Returns (str) like '1,000,000,000'
    """
    return f"{large_number:,.0f}"


# Usage

## Querying the Disinformation Tweets Table

Counting number of users and tweets:

In [80]:
print("------------")
print("QUERY:")
sql = """
    SELECT 
        count(distinct status_id) as status_count 
        ,count(distinct user_id) as user_count
    FROM `tweet-research-shared.disinfo_2021.tweets_view` t
"""

results = list(bq_service.execute_query(sql, verbose=True))
results = dict(results[0])
print("------------")
print("RESULTS...")
print("TWEETS:", fmt_n(results["status_count"]))
print("USERS:", fmt_n(results["user_count"]))


------------
QUERY:

    SELECT 
        count(distinct status_id) as status_count 
        ,count(distinct user_id) as user_count
    FROM `tweet-research-shared.disinfo_2021.tweets_view` t

------------
RESULTS...
TWEETS: 1,075,351
USERS: 349,590


Fetching some example tweets:

In [81]:
print("------------")
print("QUERY:")
sql = """
    SELECT *
    FROM `tweet-research-shared.disinfo_2021.tweets_view` t
    LIMIT 10
"""

records = [dict(row) for row in list(bq_service.execute_query(sql, verbose=True))]
print("------------")
print("RESULTS:" , len(records))


------------
QUERY:

    SELECT *
    FROM `tweet-research-shared.disinfo_2021.tweets_view` t
    LIMIT 10

------------
RESULTS: 10


In [82]:

from pandas import DataFrame

df = DataFrame(records)
df

,status_id,status_text,truncated,retweeted_status_id,retweeted_user_id,retweeted_user_screen_name,reply_status_id,reply_user_id,is_quote,geo,created_at,user_id,user_name,user_screen_name,user_description,user_location,user_verified,user_created_at
0,1354593218223104003,RT @BigBlueWaveUSA: #MarjorieTaylorGreene is e...,False,1354592005645451266,979212105533440000,BigBlueWaveUSA,None,None,False,None,2021-01-28 00:52:45+00:00,727637976377864192,DREW,drew4711700,ABOLITIONIST $DREW1700 Iraq War VET/WARHORSE,IN THE WORLD,False,2016-05-03 23:16:27+00:00
1,1354552038873575426,RT @MalcolmNance: VIDEO: Dutch Q-Anon/anti-vax...,False,1354344326931877890,2573480784,MalcolmNance,None,None,False,None,2021-01-27 22:09:07+00:00,697442340659556353,Victor Collazo,JR23910,"Free spirit, animal lover, follow politics, lo...",United States,False,2016-02-10 15:29:46+00:00
2,1354516279483371522,RT @NHarris956: A reporter asked Jen Psaki dur...,False,1354495623333109764,736166748,NHarris956,None,None,False,None,2021-01-27 19:47:01+00:00,2846554691,Faridah,faridahmalik55,"“ J’ai Baisé Ta Bouche, Iokanaan “",None,False,2014-10-27 11:22:39+00:00
3,1354519570334117888,"RT @RubenBolling: ""Goya board silences its CEO...",False,1354190345328881665,17212474,RubenBolling,None,None,False,None,2021-01-27 20:00:05+00:00,89985399,iMarcosSoup🏳️‍🌈,markcsoup,Disfruta la vida al máximo! Soy mejor de lo qu...,México / USA,False,2009-11-14 17:55:52+00:00
4,1354577795175903236,"RT @jstxvbz: ""Starting strong is good. Finishi...",False,1353879991356973057,1353503194693963788,jstxvbz,None,None,False,None,2021-01-27 23:51:27+00:00,1348838681218572291,DEPLORABLE CHUMP🇺🇸,jojodan11592387,LOVE MY COUNTRY❤️TRUE TO AMERICAN VALUES❤️TRUE...,None,False,2021-01-12 03:47:01+00:00
5,1354556968514404352,RT @OliviaTroye: Glad @DHSgov is finally allow...,False,1354556505127866370,2329492448,OliviaTroye,None,None,False,None,2021-01-27 22:28:42+00:00,2573493463,Delaney Madison @🏠🌊,DMGrill,Graphic/Fine artist. Married-no flirty DMs. Po...,"Upper Left Coast, USA ☔️💦",False,2014-06-17 19:52:59+00:00
6,1354556633708302341,RT @OliviaTroye: Glad @DHSgov is finally allow...,False,1354556505127866370,2329492448,OliviaTroye,None,None,False,None,2021-01-27 22:27:22+00:00,977727697018281984,sthitchtwit,sthitchtwit,Worked for a long time. Now happily retired. N...,None,False,2018-03-25 02:03:43+00:00
7,1354558578963738626,RT @OliviaTroye: Glad @DHSgov is finally allow...,False,1354556505127866370,2329492448,OliviaTroye,None,None,False,None,2021-01-27 22:35:06+00:00,820255908617920512,Dan Davis,Bindlestaff,Character is like a tree and reputation like a...,"Georgia, USA",False,2017-01-14 13:07:03+00:00
8,1354536363673194498,RT @gmanhum: @starkrob21 @realMikeLindell Stor...,False,1354499452493561864,275838543,gmanhum,None,None,False,None,2021-01-27 21:06:49+00:00,890727359011225600,Pres. Elect_Doctor BOB,MysterBobC,"USMC Veteran, Marine Air Surveillance/Air Defe...","Ohio, USA",False,2017-07-28 00:15:26+00:00
9,1354557626873311233,RT @OliviaTroye: Glad @DHSgov is finally allow...,False,1354556505127866370,2329492448,OliviaTroye,None,None,False,None,2021-01-27 22:31:19+00:00,16367618,Smookbert,Smookbert,None,None,False,2008-09-19 18:53:15+00:00


In [83]:
df.columns.tolist()

['status_id',
 'status_text',
 'truncated',
 'retweeted_status_id',
 'retweeted_user_id',
 'retweeted_user_screen_name',
 'reply_status_id',
 'reply_user_id',
 'is_quote',
 'geo',
 'created_at',
 'user_id',
 'user_name',
 'user_screen_name',
 'user_description',
 'user_location',
 'user_verified',
 'user_created_at']

In [84]:
df.dtypes

status_id                                  object
status_text                                object
truncated                                    bool
retweeted_status_id                        object
retweeted_user_id                          object
retweeted_user_screen_name                 object
reply_status_id                            object
reply_user_id                              object
is_quote                                     bool
geo                                        object
created_at                    datetime64[ns, UTC]
user_id                                    object
user_name                                  object
user_screen_name                           object
user_description                           object
user_location                              object
user_verified                                bool
user_created_at               datetime64[ns, UTC]
dtype: object

In [85]:
df.iloc[0]

status_id                                                   1354593218223104003
status_text                   RT @BigBlueWaveUSA: #MarjorieTaylorGreene is e...
truncated                                                                 False
retweeted_status_id                                         1354592005645451266
retweeted_user_id                                            979212105533440000
retweeted_user_screen_name                                       BigBlueWaveUSA
reply_status_id                                                            None
reply_user_id                                                              None
is_quote                                                                  False
geo                                                                        None
created_at                                            2021-01-28 00:52:45+00:00
user_id                                                      727637976377864192
user_name                               